In [119]:
import torch
from torch import nn
import torch.nn.functional as F
import time
#import networks



class Transformer():
  """
  An abstract father class for the Transformers.
  Wasn't necessary but I think it makes the code tidy
  and raises exceptions if we forget to implement something.

  Each transformer is basically just a container for the matrices needed
  for backsubstitution. Making them nn.Modules as well doesn't seem
  to me as the most intuitive choice, but that can be changed
  """

  def __init__(self):
    raise Exception("__init__ method needs to be implemented for each Transformer subclass")

  def create_backsub_matrices(self, clb: torch.Tensor, cub: torch.Tensor):
    """
      this second method is needed since some tranformers, like the leaky relu one,
      needs to know the actual concrete lowe and upper bounds to compute
      the matrices needed during backsobstitution, in addition to the layer's
      attribute
    """
    raise Exception("create_backsub_matrices method needs to be implemented for each Transformer subclass")



class LinearTransformer(Transformer):

  def __init__(self, layer : nn.Linear):
    self.weight = layer.weight.clone().detach_()
    self.bias = layer.bias.clone().detach_()

    #this two are to be used when they are the first to be multiplied in
    #backsub, that is, when the layer they are associated with is the current
    #layer we backsubstitute to
    self.weight_cat_bias_lb = torch.cat([self.weight, torch.unsqueeze(self.bias, dim = 1)], dim = 1)
    self.weight_cat_bias_ub = self.weight_cat_bias_lb


    row = torch.zeros(self.weight_cat_bias_lb.shape[1])
    row[-1] = 1
    row = torch.unsqueeze(row, dim = 0)

    #while this to are to be used when they are not associated
    #with the current layer, and so we add the 0000...01 row
    self.weight_cat_bias_backsub_lb = torch.cat([self.weight_cat_bias_lb, row], dim = 0)
    self.weight_cat_bias_backsub_ub = self.weight_cat_bias_backsub_lb

  def create_backsub_matrices(self, clb: torch.Tensor, cub: torch.Tensor):
    pass


class Conv2dTransformer(Transformer):

  def __init__(self, layer : nn.Conv2d):
    raise Exception("The Conv2dTransformer has not been implemented yet")


  def create_backsub_matrices(self, clb: torch.Tensor, cub: torch.Tensor):
    pass


class BasicReluTransformer(Transformer):

  def __init__(self, layer: nn.ReLU):
    self.weight_cat_bias_lb = None
    self.weight_cat_bias_ub = None
    self.weight_cat_bias_backsub_lb = None
    self.weight_cat_bias_backsub_ub = None

  def compute_lambda_and_intercept(lb: float, ub: float):
    """
      this is a utility method that computes the slope (lambda) and the
      intercept of the line that constitute the upper constraint when the relu
      is crossing
    """
    lambda_ = ub/(ub-lb)
    return lambda_, -(lambda_ * lb)

  def create_backsub_matrices(self, clb: torch.Tensor, cub: torch.Tensor):


    diag_entries_lb = torch.zeros_like(clb)
    diag_entries_ub = torch.zeros_like(cub)
    bias_ub = torch.zeros_like(cub)
    bias_lb = torch.zeros_like(clb)

    for i in range(len(clb)):
      if cub[i] <= 0:
        diag_entries_lb[i] = 0
        diag_entries_ub[i] = 0
      elif clb[i] >= 0 :
        diag_entries_lb[i] = 1
        diag_entries_ub[i] = 1
      else:
        diag_entries_lb[i] = 0
        lambda_ , intercept = BasicReluTransformer.compute_lambda_and_intercept(clb[i], cub[i])
        diag_entries_ub[i] = lambda_
        bias_ub[i] = intercept

    self.weight_lb = torch.diag(diag_entries_lb)
    self.weight_ub = torch.diag(diag_entries_ub)

    self.weight_cat_bias_lb = torch.cat([self.weight_lb, torch.unsqueeze(bias_lb, dim = 1)], dim = 1)
    self.weight_cat_bias_ub = torch.cat([self.weight_ub, torch.unsqueeze(bias_ub, dim = 1)], dim = 1)


    row = torch.zeros(self.weight_cat_bias_lb.shape[1])
    row[-1] = 1
    row = torch.unsqueeze(row, dim = 0)
    self.weight_cat_bias_backsub_lb = torch.cat([self.weight_cat_bias_lb, row], dim = 0)
    self.weight_cat_bias_backsub_ub = torch.cat([self.weight_cat_bias_ub, row], dim = 0)



class BasicLeakyReluTransformer(Transformer):

  def __init__(self, layer : nn.LeakyReLU):

    self.negative_slope = layer.negative_slope
    self.weight_cat_bias_lb = None
    self.weight_cat_bias_ub = None
    self.weight_cat_bias_backsub_lb = None
    self.weight_cat_bias_backsub_ub = None

  def compute_lambda_and_intercept(self, lb: float, ub: float):
    """
      this is a utility method that computes the slope (lambda) and the
      intercept of the line that constitute the upper constraint when the leaky
      relu is crossing
    """
    lambda_ = (ub - (lb * self.negative_slope))/(ub-lb)
    intercept = ((self.negative_slope - 1) * ub * lb) / ( ub - lb)
    return lambda_, intercept

  def create_backsub_matrices(self, clb: torch.Tensor, cub: torch.Tensor):


    diag_entries_lb = torch.zeros_like(clb)
    diag_entries_ub = torch.zeros_like(cub)
    bias_ub = torch.zeros_like(cub)
    bias_lb = torch.zeros_like(clb)

    for i in range(len(clb)):
      if cub[i] <= 0:
        diag_entries_lb[i] = clb[i]*self.negative_slope
        diag_entries_ub[i] = cub[i]*self.negative_slope
      elif clb[i] >= 0 :
        diag_entries_lb[i] = 1
        diag_entries_ub[i] = 1
      else:
        diag_entries_lb[i] = clb[i]*self.negative_slope
        lambda_ , intercept = self.compute_lambda_and_intercept(clb[i], cub[i])
        diag_entries_ub[i] = lambda_
        bias_ub[i] = intercept

    self.weigth_lb = torch.diag(diag_entries_lb)
    self.weigth_ub = torch.diag(diag_entries_ub)

    self.weight_cat_bias_lb = torch.cat([self.weight_lb, torch.unsqueeze(bias_lb, dim = 1)], dim = 1)
    self.weight_cat_bias_ub = torch.cat([self.weight_ub, torch.unsqueeze(bias_ub, dim = 1)], dim = 1)


    row = torch.zeros(self.weight_cat_bias_lb.shape[1])
    row[-1] = 1
    row = torch.unsqueeze(row, dim = 0)
    self.weight_cat_bias_backsub_lb = torch.cat([self.weight_cat_bias_lb, row], dim = 0)
    self.weight_cat_bias_backsub_ub = torch.cat([self.weight_cat_bias_ub, row], dim = 0)



class LeakyReluTransformer(Transformer):
  """
    For now an early attempt at implementing a tranformer that allows us to tune
    the slopes (alphas) of the lower constrain lines for when the leaky relus are
    crossing
  """

  def __init__(self, layer : nn.LeakyReLU, width: int):

    self.negative_slope = layer.negative_slope
    self.alphas = torch.zeros(width) + self.negative_slope


  def compute_lambda_and_intercept(self, lb: float, ub: float):
    lambda_ = (ub - (lb * self.negative_slope))/(ub-lb)
    intercept = ((self.negative_slope - 1) * ub * lb) / ( ub - lb)
    return lambda_, intercept

  def create_backsub_matrices(self, clb: torch.Tensor, cub: torch.Tensor):


    diag_entries_lb = torch.zeros_like(clb)
    diag_entries_ub = torch.zeros_like(cub)
    bias_ub = torch.zeros_like(cub)
    bias_lb = torch.zeros_like(clb)

    for i in range(len(clb)):
      if cub[i] <= 0:
        diag_entries_lb[i] = clb[i]*self.negative_slope
        diag_entries_ub[i] = cub[i]*self.negative_slope
      elif clb[i] >= 0 :
        diag_entries_lb[i] = 1
        diag_entries_ub[i] = 1
      else:
        diag_entries_lb[i] = clb[i]*self.alphas[i]
        lambda_ , intercept = self.compute_lambda_and_intercept(clb[i], cub[i])
        diag_entries_ub[i] = lambda_
        bias_ub[i] = intercept

    self.weigth_lb = torch.diag(diag_entries_lb)
    self.weigth_ub = torch.diag(diag_entries_ub)

    self.weight_cat_bias_lb = torch.cat([self.weight_lb, torch.unsqueeze(bias_lb, dim = 1)], dim = 1)
    self.weight_cat_bias_ub = torch.cat([self.weight_ub, torch.unsqueeze(bias_ub, dim = 1)], dim = 1)


    row = torch.zeros(self.weight_cat_bias_lb.shape[1])
    row[-1] = 1
    row = torch.unsqueeze(row, dim = 0)
    self.weight_cat_bias_backsub_lb = torch.cat([self.weight_cat_bias_lb, row], dim = 0)
    self.weight_cat_bias_backsub_ub = torch.cat([self.weight_cat_bias_ub, row], dim = 0)



class VerificationTransformer(Transformer):
  """
    The final transformer of the verifying network. It computes the 9
    differences of the logits corresponding to the true label of the sample
    and all other 9 logits
  """

  def __init__(self):

    self.weight_cat_bias_lb = torch.Tensor([[1.,-1., 0.]])
    self.weight_cat_bias_ub = self.weight_cat_bias_lb


    row = torch.zeros(self.weight_cat_bias_lb.shape[1])
    row[-1] = 1
    row = torch.unsqueeze(row, dim = 0)

    #actually don't need this, just added for "symmetry"
    self.weight_cat_bias_backsub_lb = torch.cat([self.weight_cat_bias_lb, row], dim = 0)
    self.weight_cat_bias_backsub_ub = self.weight_cat_bias_backsub_lb

  def create_backsub_matrices(self, clb: torch.Tensor, cub: torch.Tensor):
    pass



class TransformerNet(nn.Module):

  """
    The object we push the initial lb and ub through
  """

  def __init__(self, net: nn.Module, verbose = False):

    """
      This constructor scans the modules of the original network and builds a
      list of corresponding tansformers
    """
    super().__init__()
    self.verbose = verbose
    self.transformers_list = []
    self.clb_list = []
    self.cub_list = []
    self.backsub_matrices_lb = []
    self.backsub_matrices_ub = []

    if self.verbose:
      print("-"*40 + "__init__"+"-"*40+"\n")

    #copied from the repo
    #net_layers = [layer for layer in net.modules() if type(layer) not in [networks.FullyConnected, networks.Conv, nn.Sequential]]
    net_layers = [layer for layer in net.modules() if type(layer) not in [ nn.Sequential]]

    for i,layer in enumerate(net_layers):
      #if type(layer) == networks.Normalization:
          #pass
      if type(layer) == nn.Flatten:
          pass
      elif type(layer) == nn.Linear:
          self.transformers_list.append(LinearTransformer(layer))
      elif type(layer) == nn.ReLU:
          self.transformers_list.append(BasicReluTransformer(layer))
      elif type(layer) == nn.LeakyReLU:
          self.transformers_list.append(BasicLeakyReluTransformer(layer))
      elif type(layer) == nn.Conv2d:
          self.transformers_list.append(Conv2dTransformer(layer))
      else:
          raise Exception("Layer "+ str(type(layer))+ "has not a corresponding transformer yet")


      if self.verbose:
        print("Transformer n. {}: ".format(i+1)
              + str(type(self.transformers_list[-1])))
        print()
        print("weight_cat_bias_lb: ")
        print(self.transformers_list[-1].weight_cat_bias_lb)
        print()
        print("weight_cat_bias_ub: ")
        print(self.transformers_list[-1].weight_cat_bias_ub)
        print()
        print("weight_cat_bias_backsub_lb: ")
        print(self.transformers_list[-1].weight_cat_bias_backsub_lb)
        print()
        print("weight_cat_bias_backsub_ub: ")
        print(self.transformers_list[-1].weight_cat_bias_backsub_ub)
        print()
        print("-"*88+"\n")

    self.transformers_list.append(VerificationTransformer())

    if self.verbose:
        print("Verification Transformer: "
              + str(type(self.transformers_list[-1])))
        print()
        print("weight_cat_bias_lb: ")
        print(self.transformers_list[-1].weight_cat_bias_lb)
        print()
        print("weight_cat_bias_ub: ")
        print(self.transformers_list[-1].weight_cat_bias_ub)
        print()
        print("weight_cat_bias_backsub_lb: ")
        print(self.transformers_list[-1].weight_cat_bias_backsub_lb)
        print()
        print("weight_cat_bias_backsub_ub: ")
        print(self.transformers_list[-1].weight_cat_bias_backsub_ub)
        print()
        print("-"*36+"__init__ completed"+"-"*36+"\n\n\n")






  def forward(self, clb: torch.Tensor, cub: torch.Tensor):

    """
      this implementation assumes that we always want to backsobstitute to the
      input "layer", and peforms the full chain of matrix multiplications anew
      for each layer
    """
    self.starting_time = time.time()

    if self.verbose:
      print("-"*40+"forward"+"-"*40+"\n")
      print("Input lb:")
      print(clb)
      print()
      print("Input ub:")
      print(cub)
      print()
      print("-"*88+"\n")

    #these list contain the concrete ub and lb for each layer
    #the first entry is the initial tensor of lb and ub passed as arguments
    #maybe we only need the concrete constrains of last layer, but
    #keeping everything might be useful for debugging

    self.clb_list.append(clb)
    self.cub_list.append(cub)

    #these contain the matrices necessary to perform backsub
    #backsub is performed by multiplying the matrices bacwards without
    #transposing them
    self.backsub_matrices_lb.append(torch.cat([clb, torch.ones(1)], dim = 0))
    self.backsub_matrices_ub.append(torch.cat([cub, torch.ones(1)], dim = 0))

    if self.verbose:
      print("Layer 0 (before first layer of the network)\n")
      print("clb_list:")
      print(self.clb_list)
      print()
      print("cub_list:")
      print(self.cub_list)
      print()
      print("backsub_matrices_lb:")
      print(self.backsub_matrices_lb[-1])
      print()
      print("backsub_matrices_ub:")
      print(self.backsub_matrices_ub[-1])
      print()
      print("-"*88+"\n")





    for n,transformer in enumerate(self.transformers_list):



      #backsubstitution, in all its glory and splendor

      #fetch the backsub matrices of the current transformer, the one without
      #the extra 00...001 row
      transformer.create_backsub_matrices(self.clb_list[-1], self.cub_list[-1] )
      weight_cat_bias_lb = transformer.weight_cat_bias_lb
      weight_cat_bias_ub = transformer.weight_cat_bias_ub

      new_clb = weight_cat_bias_lb

      new_cub = weight_cat_bias_ub


      #to the mat multiplicattion to the very beginning
      for i in range( len(self.backsub_matrices_lb)-1,-1, -1):

        current_weight_cat_bias_backsub_lb = self.backsub_matrices_lb[i]

        current_weight_cat_bias_backsub_ub = self.backsub_matrices_ub[i]

        new_clb_positive = F.relu(new_clb)

        new_clb_negative = F.relu(-new_clb)

        new_cub_positive = F.relu(new_cub)

        new_cub_negative = F.relu(-new_cub)

        new_clb = torch.matmul(new_clb_positive, current_weight_cat_bias_backsub_lb ) - torch.matmul(new_clb_negative, current_weight_cat_bias_backsub_ub )

        new_cub = torch.matmul(new_cub_positive, current_weight_cat_bias_backsub_ub ) - torch.matmul(new_cub_negative, current_weight_cat_bias_backsub_lb )


      #add the newly computed concrete bounds to the list
      self.clb_list.append(new_clb)
      self.cub_list.append(new_cub)

      #add the backsub matrices to the list, this time the version enhanced with
      #the extra 000...0001 row
      weight_cat_bias_backsub_lb = transformer.weight_cat_bias_backsub_lb
      weight_cat_bias_backsub_ub = transformer.weight_cat_bias_backsub_ub

      self.backsub_matrices_lb.append(weight_cat_bias_backsub_lb)
      self.backsub_matrices_ub.append(weight_cat_bias_backsub_ub)

      if self.verbose:
        print("Layer {}\n".format(n+1))
        print("clb_list:")
        print(self.clb_list)
        print()
        print("cub_list:")
        print(self.cub_list)
        print()
        print("backsub_matrices_lb:")
        for mat in self.backsub_matrices_lb:
          print(mat)
          print()
        print("\n")
        print("backsub_matrices_ub:")
        for mat in self.backsub_matrices_ub:
          print(mat)
          print()
        print()

        print("-"*88+"\n")


    if self.verbose:
      print("-"*35+"forward completed"+"-"*35+"\n")
      print("Final concrete lb:")
      print(self.clb_list[-1])
      print()
      print("Total time: {} sec".format(time.time()-self.starting_time))
      print()
      print("-"*88+"\n")


    return self.clb_list[-1], self.cub_list[-1]








## Build the toy networks in slide 22 in LECTURE4_DEEPPOLY slides

In [109]:
#create the toy network


layer_1 = nn.Linear(2,2)
layer_2 = nn.ReLU()
layer_3 = nn.Linear(2,2)
layer_4 = nn.ReLU()
layer_5 = nn.Linear(2,2)


In [110]:
with torch.no_grad():
    layer_1.weight = nn.Parameter(torch.Tensor([[1.,1.],[1.,-1.]]))
    layer_1.bias = nn.Parameter(torch.zeros_like(layer_1.bias))
    layer_3.weight = nn.Parameter(torch.Tensor([[1.,1.],[1.,-1.]]))
    layer_3.bias = nn.Parameter(torch.Tensor([-0.5, 0.]))
    layer_5.weight = nn.Parameter(torch.Tensor([[-1.,1.],[0.,1.]]))
    layer_5.bias = nn.Parameter(torch.Tensor([3., 0.]))


In [111]:
net = nn.Sequential(layer_1, layer_2, layer_3, layer_4, layer_5)

In [112]:
net

Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): ReLU()
  (2): Linear(in_features=2, out_features=2, bias=True)
  (3): ReLU()
  (4): Linear(in_features=2, out_features=2, bias=True)
)

In [113]:
net.state_dict()

OrderedDict([('0.weight',
              tensor([[ 1.,  1.],
                      [ 1., -1.]])),
             ('0.bias', tensor([0., 0.])),
             ('2.weight',
              tensor([[ 1.,  1.],
                      [ 1., -1.]])),
             ('2.bias', tensor([-0.5000,  0.0000])),
             ('4.weight',
              tensor([[-1.,  1.],
                      [ 0.,  1.]])),
             ('4.bias', tensor([3., 0.]))])

## Now let's test the TransformerNet

In [114]:
transformerNet = TransformerNet(net, verbose = True )

----------------------------------------__init__----------------------------------------

Transformer n. 1: <class '__main__.LinearTransformer'>

weight_cat_bias_lb: 
tensor([[ 1.,  1.,  0.],
        [ 1., -1.,  0.]])

weight_cat_bias_ub: 
tensor([[ 1.,  1.,  0.],
        [ 1., -1.,  0.]])

weight_cat_bias_backsub_lb: 
tensor([[ 1.,  1.,  0.],
        [ 1., -1.,  0.],
        [ 0.,  0.,  1.]])

weight_cat_bias_backsub_ub: 
tensor([[ 1.,  1.,  0.],
        [ 1., -1.,  0.],
        [ 0.,  0.,  1.]])

----------------------------------------------------------------------------------------

Transformer n. 2: <class '__main__.BasicReluTransformer'>

weight_cat_bias_lb: 
None

weight_cat_bias_ub: 
None

weight_cat_bias_backsub_lb: 
None

weight_cat_bias_backsub_ub: 
None

----------------------------------------------------------------------------------------

Transformer n. 3: <class '__main__.LinearTransformer'>

weight_cat_bias_lb: 
tensor([[ 1.0000,  1.0000, -0.5000],
        [ 1.0000, -

In [115]:
#define the input tensors as in the slide

lb = torch.Tensor([-1., -1.])
ub = torch.Tensor([1., 1.])

In [116]:
final_lb, final_ub = transformerNet(lb, ub)

----------------------------------------forward----------------------------------------

Input lb:
tensor([-1., -1.])

Input ub:
tensor([1., 1.])

----------------------------------------------------------------------------------------

Layer 0 (before first layer of the network)

clb_list:
[tensor([-1., -1.])]

cub_list:
[tensor([1., 1.])]

backsub_matrices_lb:
tensor([-1., -1.,  1.])

backsub_matrices_ub:
tensor([1., 1., 1.])

----------------------------------------------------------------------------------------

Layer 1

clb_list:
[tensor([-1., -1.]), tensor([-2., -2.])]

cub_list:
[tensor([1., 1.]), tensor([2., 2.])]

backsub_matrices_lb:
tensor([-1., -1.,  1.])

tensor([[ 1.,  1.,  0.],
        [ 1., -1.,  0.],
        [ 0.,  0.,  1.]])



backsub_matrices_ub:
tensor([1., 1., 1.])

tensor([[ 1.,  1.,  0.],
        [ 1., -1.,  0.],
        [ 0.,  0.,  1.]])


----------------------------------------------------------------------------------------

Layer 2

clb_list:
[tensor([-1.,

In [120]:
#just for curisosity, test the time for forward without debug prints
transformerNet = TransformerNet(net, verbose = False )
lb = torch.Tensor([-1., -1.])
ub = torch.Tensor([1., 1.])
start_time = time.time()
final_lb, final_ub = transformerNet(lb, ub)
print("Time without debug prints: {}".format(time.time()-start_time))


Time without debug prints: 0.0024890899658203125
